# Selenium 만으로 유튜브 랭킹 수집하기

In [ ]:
!apt-get update > /dev/null
!pip install selenium > /dev/null
!apt install chromium-chromedriver > /dev/null

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [4]:
'''options = webdriver.ChromeOptions()
options.add_argument('--headless')   # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")'''
driver = webdriver.Chrome('C:/workspace/chromedriver/chromedriver.exe')  # 인수로 한나더  ,options=options해야됨

In [5]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=1'
driver.get(url)

In [7]:
# pop-up 창이 있으면, pop-up 창을 닫고 메인 윈도우로 되돌아 옴
window_len = len(driver.window_handles)
if window_len == 2:
    driver.switch_to.window(driver.window_handles[1])
    driver.close()

# 화면의 끝까지 PageDown으로 내려보냄
driver.switch_to.window(driver.window_handles[0])
body = driver.find_element_by_css_selector('body')
for i in range(30):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

KeyboardInterrupt: 

In [8]:
trs = driver.find_elements_by_css_selector('.aos-init')
len(trs)

100

In [9]:
trs[99].text

'99\n[BJ/인물/연예인]\n허팝Heopop 0\n허팝 Heopop HeopopTV\n381만 31억4525만 1,850개 6,118'

In [10]:
def convert_unit(s):
    #s = ''.join(s.split('억'))
    s = s.replace('억', '').replace('개','').replace(',','')
    s = s.replace('만', '0000')
    return f'{int(s):,d}'

In [12]:
tr = trs[99]
category = tr.find_element_by_css_selector('p.category').text.strip(' \n[]')
category

'BJ/인물/연예인'

In [13]:
name = tr.find_element_by_css_selector('.subject a').text.strip()
subscriber = convert_unit(tr.find_element_by_css_selector('.subscriber_cnt').text)
view = convert_unit(tr.find_element_by_css_selector('.view_cnt').text)
video = convert_unit(tr.find_element_by_css_selector('.video_cnt').text)
name, subscriber, view, video

('허팝Heopop', '3,810,000', '3,145,250,000', '1,850')

In [14]:
from tqdm.notebook import tqdm

In [ ]:
results = []
for page in tqdm(range(1, 11)):
    #print(page)
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+str(page)
    driver.get(url)
    time.sleep(2)

    window_len = len(driver.window_handles)
    if window_len == 2:
        driver.switch_to.window(driver.window_handles[1])
        driver.close()
    driver.switch_to.window(driver.window_handles[0])
    body = driver.find_element_by_css_selector('body')
    for i in range(30):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
    
    trs = driver.find_elements_by_css_selector('.aos-init')

    for tr in trs:
        category = tr.find_element_by_css_selector('p.category').text.strip(' \n[]')
        #category = channel.select_one('p.category').get_text().strip(' \n[]')
        name = tr.find_element_by_css_selector('.subject a').text.strip()
        #name = channel.select_one('.subject a').text.strip()
        subscriber = convert_unit(tr.find_element_by_css_selector('.subscriber_cnt').text)
        #subscriber = convert_unit(channel.select_one('.subscriber_cnt').text)
        view = convert_unit(tr.find_element_by_css_selector('.view_cnt').text)
        #view = convert_unit(channel.select_one('.view_cnt').text)
        video = convert_unit(tr.find_element_by_css_selector('.video_cnt').text)
        #video = convert_unit(channel.select_one('.video_cnt').text)
        results.append([category,name,subscriber,view,video])
